# Demonstration of NN-Equivalence Tool #

In this notebook, we run through an example of verifying equivalence for two neural networks.

In [3]:
from performance import Encoder

Example neural networks are stored in the `ExampleNNs` directory.
We use the balance scale network, because they are very small and therefore their encodings are not too long, so we can print them.

In this case the test network is a version of the reference network with the value of few weights being manually altered.

In [4]:
reference = 'ExampleNNs/balance_scale_lin.h5'
test = 'ExampleNNs/balance_scale_lin2.h5'

We want to verify top-2-equivalence.

The mode is named `one_hot_partial_top_2`, because we use a partial permutation matrix to retrieve activations of the 2 most likely classes from the test network.
For the reference network, the partial permutation matrix needed to retrieve only the top output has only one row and is therefore a one hot vector.

For mode-names corresponding to top-k-equivalence for other values of k, the number at the end of the mode-name can be changed.

For $\varepsilon$-equivalence, modes `optimize_diff_[manhattan | chebyshev]` can be used.

In [5]:
mode = 'one_hot_partial_top_2'

We need to specify lower and upper bounds on the inputs.

In [6]:
input_los = [0,0,0,0]
input_his = [5,5,5,5]

Instantiate an `Encoder` object and tell it to encode equivalence between the reference and the test network.

I did not remove the debug output for the cell below. It can be ignored.

In [7]:
enc = Encoder()
enc.encode_equiv(reference, test, input_los, input_his, mode)

### Vars ###
### Constraints ###
A_pi_3_0 = 0 --> (A_x_2_0 + 0.0001) <= A_top_3_0
A_pi_3_1 = 0 --> (A_x_2_1 + 0.0001) <= A_top_3_0
A_pi_3_2 = 0 --> (A_x_2_2 + 0.0001) <= A_top_3_0


We want to restrict the input-domain for the equivalence verification to points within a radius of 5, measured in the manhattan distance.

In [8]:
center = [5,5,0,0]
radius = 5

enc.add_input_radius(center, radius, metric='manhattan')

We print the encoding of this verification instance.

The first section of the output contains the variables of the encoding with intervals describing their lower and upper bounds.
Note, that all variables other than the input variables are set to default bounds, as we didn't perform bounds optimization yet.

Every variable has a (sometimes empty) prefix, indicating, to which network it belongs.
* Variables belonging to the input layer are only encoded once and don't have a net-prefix.
* Variables belonging to the reference network are prefixed with 'A'
* Variables belonging to the test network are prefixed with 'B'
* Variables belonging to the encoding of the equivalence property are prefixed with 'E'

Furthermore every variable has two indices.
Variable `A_x_0_3` for example belongs to the first (the 0th) layer of the reference network and is in the 4th row of that layer.
For variables not belonging to layered architecture of a network, the indices don't have a special meaning.

Variables named `i` are input variables, while variables named `x` typically represent linear combinations of values of neurons in the preceding layer.
Variables named `d` represent binary variables.

Later we are interested in maximizing the variable `E_diff_0_2`, which contains the violation value for our equivalence property.

Constraints can be found in the second section of the output.

In [9]:
enc.pretty_print()

### Vars ###
i_0_0: [0, 5]
i_0_1: [0, 5]
i_0_2: [0, 5]
i_0_3: [0, 5]
d_0_0: [0, 1]
d_0_1: [0, 1]
d_0_2: [0, 1]
d_0_3: [0, 1]
abs_0_0: [0, 999999]
abs_0_1: [0, 999999]
abs_0_2: [0, 999999]
abs_0_3: [0, 999999]
A_x_0_0: [-999999, 999999]
A_x_0_1: [-999999, 999999]
A_x_0_2: [-999999, 999999]
A_x_0_3: [-999999, 999999]
A_d_0_0: [0, 1]
A_d_0_1: [0, 1]
A_d_0_2: [0, 1]
A_d_0_3: [0, 1]
A_o_0_0: [0, 999999]
A_o_0_1: [0, 999999]
A_o_0_2: [0, 999999]
A_o_0_3: [0, 999999]
A_x_1_0: [-999999, 999999]
A_x_1_1: [-999999, 999999]
A_x_1_2: [-999999, 999999]
A_x_1_3: [-999999, 999999]
A_d_1_0: [0, 1]
A_d_1_1: [0, 1]
A_d_1_2: [0, 1]
A_d_1_3: [0, 1]
A_o_1_0: [0, 999999]
A_o_1_1: [0, 999999]
A_o_1_2: [0, 999999]
A_o_1_3: [0, 999999]
A_x_2_0: [-999999, 999999]
A_x_2_1: [-999999, 999999]
A_x_2_2: [-999999, 999999]
A_y_0_0: [-999999, 999999]
A_y_1_0: [-999999, 999999]
A_y_2_0: [-999999, 999999]
A_top_3_0: [-999999, 999999]
A_pi_3_0: [0, 1]
A_pi_3_1: [0, 1]
A_pi_3_2: [0, 1]
B_x_0_0: [-999999, 999999]
B_x_0_1: [

Now we perform bounds tightening by interval arithmetic and by solving smaller sub-problems.

The output is the Gurobi-log for the subproblems solved.

In [10]:
enc.optimize_constraints()

Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 20.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 20.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 29 rows, 17 columns and 73 nonzeros
Variable types: 13 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [2e-02, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [2e-02, 7e+00]
Presolve removed 9 rows and 1 columns
Presolve time: 0.00s
Presolved: 20 rows, 16 columns, 56 nonzeros
Variable types: 12 continuous, 4 integer (4 binary)

Root relaxation: objective 5.764081e+00, 9 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.76408    0    3          -    5.76408      -     -    0s
H    0     0                  


Root relaxation: objective 4.488114e+00, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.48811    0    1          -    4.48811      -     -    0s
H    0     0                       4.4843848    4.48811  0.08%     -    0s
     0     0     cutoff    0         4.48438    4.48438  0.00%     -    0s

Explored 1 nodes (5 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 4.48438 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.484384754976e+00, best bound 4.484384754976e+00, gap 0.0000%
Optimize a model with 29 rows, 17 columns and 73 nonzeros
Variable types: 13 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [2e-02, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [9e-02, 7e+00]
Presolve removed 9 rows and 1

  Bounds range     [3e-01, 8e+00]
  RHS range        [2e-02, 8e+00]
Presolve removed 9 rows and 1 columns
Presolve time: 0.00s
Presolved: 20 rows, 16 columns, 56 nonzeros
Variable types: 12 continuous, 4 integer (4 binary)

Root relaxation: objective -7.300947e-01, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      -0.7300947   -0.73009  0.00%     -    0s

Explored 0 nodes (11 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: -0.730095 

Optimal solution found (tolerance 1.00e-04)
Best objective -7.300947135949e-01, best bound -7.300947135949e-01, gap 0.0000%
Changed value of parameter TimeLimit to 20.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 20.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model 

     0     0    0.11973    0    1    0.18601    0.11973  35.6%     -    0s
*    0     0               0       0.1788253    0.17883  0.00%     -    0s

Cutting planes:
  Gomory: 2
  Implied bound: 2
  MIR: 2

Explored 1 nodes (20 simplex iterations) in 0.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: 0.178825 0.186007 0.222994 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.788253466277e-01, best bound 1.788253466277e-01, gap 0.0000%
Changed value of parameter TimeLimit to 20.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 20.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 29 rows, 17 columns and 73 nonzeros
Variable types: 13 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [9e-02, 1e+01]
Presolve removed 9 rows and 1 columns
Presolve

  RHS range        [2e-02, 9e+00]
Presolve removed 8 rows and 3 columns
Presolve time: 0.00s
Presolved: 16 rows, 14 columns, 47 nonzeros
Variable types: 11 continuous, 3 integer (3 binary)

Root relaxation: objective 1.053113e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   10.53113    0    1          -   10.53113      -     -    0s
H    0     0                      10.5299136   10.53113  0.01%     -    0s
     0     0 infeasible    0        10.52991   10.52991  0.00%     -    0s

Explored 1 nodes (6 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 10.5299 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.052991359379e+01, best bound 1.052991359379e+01, gap 0.0000%
Optimize a model with 24 rows, 17 columns and 63 nonzeros
Variable types: 13 continuous, 4 integer (4 binary)


We can see, that the bounds of the variables have been tightened.

In [11]:
enc.pretty_print()

### Vars ###
i_0_0: [0, 5]
i_0_1: [0, 5]
i_0_2: [0, 5]
i_0_3: [0, 5]
d_0_0: [0, 0]
d_0_1: [0, 0]
d_0_2: [0, 1]
d_0_3: [0, 1]
abs_0_0: [0, 5]
abs_0_1: [0, 5]
abs_0_2: [0, 5]
abs_0_3: [0, 5]
A_x_0_0: [-4.464432448148727, 2.4794387072324753]
A_x_0_1: [-3.687929444015026, 5.7785393968224525]
A_x_0_2: [-1.7777611389756203, 3.729954317212105]
A_x_0_3: [-1.7894070968031883, 7.332582697272301]
A_d_0_0: [0, 1]
A_d_0_1: [0, 1]
A_d_0_2: [0, 1]
A_d_0_3: [0, 1]
A_o_0_0: [0, 2.4794387072324753]
A_o_0_1: [0, 5.7785393968224525]
A_o_0_2: [0, 3.729954317212105]
A_o_0_3: [0, 7.332582697272301]
A_x_1_0: [-1.8478958275373936, 5.076723421703085]
A_x_1_1: [-0.32485962698912885, 1.8436871195505793]
A_x_1_2: [-1.804407192748503, 8.159865636119198]
A_x_1_3: [-6.631261264248645, 4.4843847549760945]
A_d_1_0: [0, 1]
A_d_1_1: [0, 1]
A_d_1_2: [0, 1]
A_d_1_3: [0, 1]
A_o_1_0: [0, 5.076723421703085]
A_o_1_1: [0, 1.8436871195505793]
A_o_1_2: [0, 8.159865636119198]
A_o_1_3: [0, 4.4843847549760945]
A_x_2_0: [-6.915419027

Now we use Gurobi to optimize the encoding.

At the very top of the log, we can see the size of the encoding.
A few lines below is the size of the mixed integer linear program (MILP) after application of Gurobi's presolve routine.

The column `Incumbent` contains the best (in this case the maximal) value of the objective function (here the variable `E_diff_0_2` representing the violation score of top-2-equivalence).
The column `BestBd` contains bounds on this value (upper bounds for maximization problems, lower bounds for minimization problems).

The final solution is printed at the bottom.
As it is greater than zero, the test network is not top-2-equivalent to the reference network.

In [12]:
model = enc.create_gurobi_model()
model.optimize()

Optimize a model with 187 rows, 95 columns and 484 nonzeros
Variable types: 63 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 2e+01]
  RHS range        [1e-04, 2e+01]
Presolve removed 59 rows and 21 columns
Presolve time: 0.01s
Presolved: 128 rows, 74 columns, 406 nonzeros
Variable types: 52 continuous, 22 integer (22 binary)

Root relaxation: objective 1.121044e+01, 61 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.21044    0   15          -   11.21044      -     -    0s
H    0     0                       2.9512734   11.21044   280%     -    0s
     0     0    7.53680    0   16    2.95127    7.53680   155%     -    0s
     0     0    7.00562    0   16    2.95127    7.00562   137%     -    0s
     0     0    6.97588    0   17    2.9512

In [13]:
from analysis import get_grb_inputs, calculate_violation

We can now analyze the counterexample found by Gurobi.

First we retrieve the counterexample. The required method has signature `get_grb_inputs(model, input_size)`.

In [14]:
ins = get_grb_inputs(model, 4)
print(ins)

[5.0, 5.0, 5.0, 0.0]


We have a dedicated method to calculate the actual violation to top-k-equivalence.

It takes the found counterexample, the value of k and the paths to the verified neural networks as inputs.

In [15]:
calculate_violation(ins, reference, test, top_k=2)

2.9512734022428213

This time we want to find a maximal equivalence radius around our centerpoint. First we instantiate a new encoder.

In [16]:
enc_r = Encoder()
enc_r.encode_equiv(reference, test, input_los, input_his, mode)

### Vars ###
### Constraints ###
A_pi_3_0 = 0 --> (A_x_2_0 + 0.0001) <= A_top_3_0
A_pi_3_1 = 0 --> (A_x_2_1 + 0.0001) <= A_top_3_0
A_pi_3_2 = 0 --> (A_x_2_2 + 0.0001) <= A_top_3_0


Then we add an input radius, however with `radius_mode='variable'`.

We use an upper bound of 5 on our radius, since we know, that they are not equivalent for this radius from our first verification problem.

In [17]:
radius_lo = 0
radius_hi = 5

enc_r.add_input_radius(center, radius_hi, metric='manhattan', radius_lo=radius_lo, radius_mode='variable')

We can then optimize for the radius.

We omit the bounds tightening here, since the balance_scale networks are very small.

In [18]:
model_r = enc_r.create_gurobi_model()
model_r.optimize()

Optimize a model with 205 rows, 96 columns and 528 nonzeros
Variable types: 64 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e-04, 2e+06]
Presolve removed 74 rows and 22 columns
Presolve time: 0.01s
Presolved: 131 rows, 74 columns, 413 nonzeros
Variable types: 51 continuous, 23 integer (23 binary)

Root relaxation: objective 0.000000e+00, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8          -    0.00000      -     -    0s
H    0     0                       2.2720034    0.00000   100%     -    0s
     0     0    0.00000    0    8    2.27200    0.00000   100%     -    0s
     0     0    0.00000    0    7    2.27200    0.00000   100%     -    0s
     0     0    0.00000    0    7    2.2720

First, we test, if Gurobi found a valid counterexample for radius 1.99797.

As it produces a violation greater than 0.01, it is exceeds our threshold for equivalence and thus is a valid counterexample.

The equivalence threshold, along with other flags and constants can be found in the `flags_constants.py` module.

In [19]:
ins_r = get_grb_inputs(model_r, 4)
print('Counterexample: ', ins_r)

calculate_violation(ins_r, reference, test, top_k=2)

Counterexample:  [5.0, 5.0, 1.9979682622049242, 0.0]


0.010000000000001119

Now we test, if Gurobi finds, that the two networks are equivalent, when restricting the input-region to a radius just below the smallest radius, for which Gurobi still found a counterexample, whose violation was larger than 0.01.

Within a radius of 1.95, the violation of the equivalence property is indeed almost zero.

In [20]:
enc_t = Encoder()
enc_t.encode_equiv(reference, test, input_los, input_his, mode)
enc_t.add_input_radius(center, 1.95)
model_t = enc_t.create_gurobi_model()
model_t.optimize()

### Vars ###
### Constraints ###
A_pi_3_0 = 0 --> (A_x_2_0 + 0.0001) <= A_top_3_0
A_pi_3_1 = 0 --> (A_x_2_1 + 0.0001) <= A_top_3_0
A_pi_3_2 = 0 --> (A_x_2_2 + 0.0001) <= A_top_3_0
Optimize a model with 204 rows, 95 columns and 526 nonzeros
Variable types: 63 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e-04, 2e+06]
Presolve removed 142 rows and 56 columns
Presolve time: 0.01s
Presolved: 62 rows, 39 columns, 198 nonzeros
Variable types: 26 continuous, 13 integer (13 binary)

Root relaxation: objective 4.536584e+00, 38 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.53658    0   10          -    4.53658      -     -    0s
H    0     0                       0.0000000    4.53658      -     -    0s
     0     0 